<a href="https://colab.research.google.com/github/anwishn/ESAA_OB_23-1/blob/main/0515_hw_%EC%B6%94%EC%B2%9C_%EC%8B%9C%EC%8A%A4%ED%85%9C_surprise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

파이썬 머신러닝 완벽 가이드 ch9. 8 pg.607-624

## 08. 파이썬 추천 시스템 패키지 - Surprise



---



### Surprise 패키지 소개


**Surprise** : 파이썬 기반의 추천 시스템 구축을 위한 전용 패키지

* 장점
  + 다양한 추천 알고리즘 쉽게 적용해 추천 시스템 구축 가능
  + Surprise의 핵심 API는 사이킷런의 핵심 API와 유사한 API명으로 작성

In [1]:
!pip install scikit-surprise

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 kB 19.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp310-cp310-linux_x86_64.whl size=3095455 sha256=d862ac5876d274a3aca84751de3c30d4874888d3e354384bdf53a45372bfc49f
  Stored in directory: /root/.cache/pip/wheels/a5/ca/a8/4e28def53797fdc4363ca4af740db15a9c2f1595ebc51fb445
Successfully built scikit-surprise


### Surprise를 이용한 추천 시스템 구축

* 추천 데이터를 학습용과 테스트용 데이터 세트로 분리 -> SVD 행렬 분해를 통한 잠재 요인 협업 필터링 수행

In [2]:
from surprise import SVD
from surprise import Dataset
from surprise import accuracy
from surprise.model_selection import train_test_split

* 데이터 로딩은 Dataset 클래스 이용해서만 가능
* 주요 데이터가 로우 레벨 형태로 돼 있는 포맷의 데이터만 처리

In [3]:
# Surprise는 무비렌즈 사이트에서 제공하는 과거 버전의 데이터 세트 가져오는 API 제공
data = Dataset.load_builtin('ml-100k')
# 수행 시마다 동일하게 데이터를 분할하기 위해 random_state 값 부여
trainset, testset = train_test_split(data, test_size=0.25, random_state=0)

Dataset ml-100k could not be found. Do you want to download it? [Y/n] Y
Trying to download dataset from https://files.grouplens.org/datasets/movielens/ml-100k.zip...
Done! Dataset ml-100k has been saved to /root/.surprise_data/ml-100k


* Surprise에 사용자-아이템 평점 데이터를 적용할 때 주의해야 할 점

-> Surprise는 자체적으로 로우 레벨의 데이터를 칼럼 레벨의 데이터로 변경하므로 원본인 로우 레벨의 사용자-아이템 평점 데이터를 데이터 세트로 적용해야 한다

* SVD로 잠재 요인 협업 필터링 수행

In [4]:
algo = SVD()
algo.fit(trainset)

* **test()** : 사용자-아이템 평점 데이터 세트 전체에 대해서 추천을 예측하는 메서드

* **predict()** : 개별 사용자와 영화에 대한 추천 평점 반환

#### test()

In [5]:
# 테스트 데이터 세트 전체에 대해 추천 영화 평점 데이터를 생성한 뒤에 최초 5개만 추출
predictions=algo.test(testset)
print('predictions type :', type(predictions), 'size :', len(predictions))
print('prediction 결과의 최초 5개 추출')
predictions[:5]

predictions type : <class 'list'> size : 25000
prediction 결과의 최초 5개 추출


[Prediction(uid='120', iid='282', r_ui=4.0, est=3.6747556773991055, details={'was_impossible': False}),
 Prediction(uid='882', iid='291', r_ui=4.0, est=3.8133817033948514, details={'was_impossible': False}),
 Prediction(uid='535', iid='507', r_ui=5.0, est=4.032151656443451, details={'was_impossible': False}),
 Prediction(uid='697', iid='244', r_ui=5.0, est=3.5736194732416755, details={'was_impossible': False}),
 Prediction(uid='751', iid='385', r_ui=4.0, est=3.6968155508231475, details={'was_impossible': False})]

=> Prediction 객체는 Surprise패키지에서 제공하는 데이터 타입. 

=> 개별 사용자 아이디(uid), 영화 아이디(iid)와 실제 평점(r_ui) 정보에 기반해 Surprise의 추천 예측 평점(est) 데이터를 튜플 형태로 가져.

=> details 속성은 내부 처리 시 추천 예측을 할 수 없는 경우에 로그용으로 데이터를 남기는 데 사용. 'was_impossible'이 True면 예측값을 생성할 수 없는 데이터라는 의미

=> 속성에 접근하려면 객체명.uid와 같은 형식으로 가능

In [6]:
[ (pred.uid, pred.iid, pred.est) for pred in predictions[:3]]

[('120', '282', 3.6747556773991055),
 ('882', '291', 3.8133817033948514),
 ('535', '507', 4.032151656443451)]

#### predict()

In [7]:
# 인자로 개별 사용자 아이디, 아이템 아이디 입력하면 추천 예측 평점을 포함한 정보 반환
# 사용자 아이디, 아이템 아이디는 문자열로 입력해야함.
uid=str(196)
iid=str(302)
pred=algo.predict(uid, iid)
print(pred)

user: 196        item: 302        r_ui = None   est = 4.05   {'was_impossible': False}


* test()메서드는 입력 데이터 세트의 모든 사용자와 아이템 아이디에 대해서 predict()를 반복적으로 수행한 결과

In [8]:
# 테스트 데이터 세트 이용해 추천 예측 평점과 실제 평점과의 차이 평가
accuracy.rmse(predictions)

RMSE: 0.9478


0.947777514977195

### Surprise 주요 모듈 소개

#### Dataset

* Surprise는 데이터 세트의 칼럼 순서가 사용자 아이디, 아이템 아이디, 평점 순으로 된 것만 적용 가능

>* Dataset.load_builtin(name='ml-100k') : 무비렌즈 아카이브 FTP 서버에서 무비렌즈 데이터 내려받기
>* Dataset.load_from_file(file_path, reader) : OS 파일에서 데이터 로딩할 때 사용
>* Dataset.load_from_df(df, reader) : 판다스의 DataFrame에서 데이터 로딩

#### OS 파일 데이터를 Surprise 데이터 세트로 로딩

* **Dataset.load_from_file** API 이용해 지정된 디렉터리에 있는 사용자-아이템 평점 데이터 로딩
  + 주의할 점 : 로딩되는 데이터 파일에 칼럼명을 가지는 헤더 문자열이 있어서는 안 된다
  + to_csv() 함수를 이용해 칼럼 헤더 삭제

In [9]:
import pandas as pd

ratings=pd.read_csv('/content/drive/MyDrive/data/ml-latest-small/ratings.csv')
# ratings_noh.csv 파일로 언로드 시 인덱스와 헤더를 모두 제거한 새로운 파일 생성.
ratings.to_csv('/content/drive/MyDrive/data/ml-latest-small/ratings_noh.csv', index=False, header=False)

* **Reader 클래스**를 이용해 데이터 파일의 파싱 포맷을 정의해야 한다
  + 이 4개의 칼럼이 사용자 아이디, 아이템 아이디, 평점, 타임스탬프임을 로딩할 때 알려줘야 한다

* Reader 클래스의 생성자에 각 필드의 칼럼명과 칼럼 분리문자, 그리고 최소~최대 평점을 입력해 객체 생성 -> load_from_file()로 생성된 Reader 객체를 참조해 데이터 파일을 파싱하면서 로딩

In [10]:
from surprise import Reader

reader = Reader(line_format = 'user item rating timestamp', sep=',', rating_scale=(0.5,5))
data = Dataset.load_from_file('/content/drive/MyDrive/data/ml-latest-small/ratings_noh.csv', reader=reader)

* Reader 클래스의 주요 생성 파라미터
  + line_format(string) : 칼럼을 순서대로 나열. 입력된 문자열을 공백으로 분리해 칼럼으로 인식
  + sep(char) : 칼럼을 분리하는 분리자. 디폴트는 '\t'. 
  + rating_scale(tuple, optional) : 평점 값의 최소~최대 평점 설정. 디폴트는 (1,5).

* SVD 행렬 분해 기법을 이용해 추천 예측
  + 잠재 요인 크기 K값을 나타내는 파라미터 n_factors 설정

In [11]:
trainset, testset=train_test_split(data, test_size=0.25, random_state=0)

#수행시마다 동일한 결과를 도출하기 위해 random_state 설정
algo=SVD(n_factors=50, random_state=0)

#학습 데이터 세트로 학습하고 테스트 데이터 세트로 평점 예측 후 rmse 평가
algo.fit(trainset)
predictions=algo.test(testset)
accuracy.rmse(predictions)

RMSE: 0.8682


0.8681952927143516

#### 판다스 DataFrame에서 Surprise 데이터 세트로 로딩

* **Dataset.load_from_df()** 이용
* 주의할 점 : DataFrame 역시 사용자 아이디, 아이템 아이디, 평점 칼럼 순서 지켜야 한다

In [12]:
import pandas as pd
from surprise import Reader, Dataset

ratings=pd.read_csv('/content/drive/MyDrive/data/ml-latest-small/ratings.csv')
reader=Reader(rating_scale=(0.5, 5.0))

# ratings DataFrame에서 칼럼은 사용자 아이디, 아이템 아이디, 평점 순서 지켜야 한다
data=Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)
trainset, testset=train_test_split(data, test_size=0.25, random_state=0)

algo=SVD(n_factors=50, random_state=0)
algo.fit(trainset)
predictions=algo.test(testset)
accuracy.rmse(predictions)

RMSE: 0.8682


0.8681952927143516

### Surprise 추천 알고리즘 클래스

1. SVD : 행렬 분해를 통하 잠재 요인 협업 필터링을 위한 SVD 알고리즘
2. KNNBasic : 최근접 이웃 협업 필터링을 위한 KNN 알고리즘
3. BaselineOnly : 사용자 Bias와 아이템 Bias를 감안한 SGD 베이스라인 알고리즘

* Surprise SVD의 비용 함수는 사용자 베이스라인 편향성을 감안한 평점 예측에 Regularization 적용한 것
* SVD 클래스의 입력 파라미터
  + n_factors : 잠재 요인 K의 개수. 디폴트는 100. 커질수록 정확도가 높아질 수 있으나 과적합 문제 발생할 수도.
  + n_epochs : SGD 수행 시 반복 횟수. 디폴트는 20.
  + biased(bool) : 베이스라인 사용자 편향 적용 여부. 디폴트는 True


* SVD++ 알고리즘 성적 좋지만, 상대적으로 시간 너무 오래 걸려 사용 어려울 것. SVD와 k-NN Baseline 성능 평가 수치 좋아 -> Baseline 결합한 경우 성능 평가 수치 대폭 향상
  + Baseline : 각 개인이 평점을 부여하는 성향을 반영해 평점을 계산하는 방식

### 베이스라인 평점

**베이스라인 평점** : 개인의 성향을 반영해 아이템 평가에 편향성 요소를 반영하여 평점을 부과하는 것

* 보통 베이스라인 평점은 전체 평균 평점 + 사용자 편향 점수 + 아이템 편향 점수 공식으로 계산
  + 전체 평균 평점 = 모든 사용자의 아이템에 대한 평점을 평균한 값
  + 사용자 편향 점수 = 사용자별 아이템 평점 평균 값 - 전체 평균 평점
  + 아이템 편향 점수 = 아이템별 평점 평균 값 - 전체 평균 평점

### 교차 검증과 하이퍼 파라미터 튜닝

* Surprise는 교차 검증과 하이퍼 파라미터 튜닝을 위해 **cross_validate()**와 **GridSearchCV** 클래스 제공

* cross_validate() 이용해 데이터를 5개의 학습/검증 폴드 데이터 세트로 분리해 교차 검증 수행하고 RMSE, MAE로 성능 평가 진행
  + 인자로 알고리즘 객체, 데이터, 성능 평가 방법(measures), 폴드 데이터 세트 개수(cv) 입력

In [13]:
from surprise.model_selection import cross_validate

#판다스 df에서 surprise 데이터 세트로 데이터 로딩
ratings=pd.read_csv('/content/drive/MyDrive/data/ml-latest-small/ratings.csv')
reader=Reader(rating_scale=(0.5, 5.0))
data=Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)

algo=SVD(random_state=0)
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8791  0.8747  0.8779  0.8677  0.8692  0.8737  0.0046  
MAE (testset)     0.6751  0.6723  0.6728  0.6661  0.6719  0.6717  0.0030  
Fit time          3.44    3.02    1.71    2.28    1.75    2.44    0.69    
Test time         0.26    0.32    0.19    0.24    0.14    0.23    0.06    


{'test_rmse': array([0.87906335, 0.87471539, 0.87794309, 0.86769324, 0.86916608]),
 'test_mae': array([0.67512686, 0.67234518, 0.6727766 , 0.66613125, 0.67192819]),
 'fit_time': (3.4356565475463867,
  3.0190515518188477,
  1.710524082183838,
  2.279862642288208,
  1.751800298690796),
 'test_time': (0.26446986198425293,
  0.3169710636138916,
  0.18613457679748535,
  0.24051189422607422,
  0.13914251327514648)}

=> 폴드별 성능 평가 수치와 전체 폴드의 평균 성능 평가 수치 함께 보여줘

* GridSearchCV도 교차 검증을 통한 하이퍼 파라미터 최적화 수행
  + SVD의 경우 주로 점진적 하강 방식의 반복횟수를 지정하는 **n_epochs**와 SVD의 잠재 요인 K의 크기를 지정하는 **n_factors** 튜닝

In [14]:
from surprise.model_selection import GridSearchCV

#최적화할 파라미터를 딕셔너리 형태로 지정.
param_grid={'n_epochs':[20, 40, 60],
            'n_factors':[50, 100, 200]}

#CV를 3개 폴드 세트로 지정, 성능 평가를 rmse, mse로 수행하도록 GridSearchcv 구성
gs=GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=3)
gs.fit(data)

#최고 RMSE evaluation 점수와 그때의 하이퍼 파라미터
print(gs.best_score['rmse'])
print(gs.best_params['rmse'])

0.8768700923773505
{'n_epochs': 20, 'n_factors': 50}


### Surprise를 이용한 개인화 영화 추천 시스템 구축

Surprise 패키지로 학습된 추천 알고리즘을 기반으로 특정 사용자가 아직 평점을 매기지 않은 영화 중에서 개인 취향에 가장 적절한 영화 추천


In [15]:
# 다음 코드는 train_test_split()으로 분리되지 않는 데이터 세트에 fit()을 호출해 오류 발생

#data=Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)
#algo=SVD(n_factors=50, random_state=0)
#algo.fit(data)

* 데이터 세트 전체를 학습 데이터로 사용하려면 **DatasetAutoFolds** 클래스 이용해 객체 생성 -> **build_full_trainset()** 메서드 호출

In [16]:
from surprise.dataset import DatasetAutoFolds

reader=Reader(line_format='user item rating timestamp', sep=',', rating_scale=(0.5, 5))
# DatasetAutoFolds 클래스를 ratings_noh.csv 파일 기반으로 생성
data_folds=DatasetAutoFolds(ratings_file='/content/drive/MyDrive/data/ml-latest-small/ratings_noh.csv', reader=reader)

# 전체 데이터를 학습 데이터로 생성
trainset=data_folds.build_full_trainset()

* SVD 이용해 학습 수행

In [17]:
algo = SVD(n_epochs=20, n_factors=50, random_state=0)
algo.fit(trainset)

* userId 9가 아직 평점을 매기지 않은 영화를 movieId 42로 선정한 뒤 예측 평점 계산

In [18]:
# 영화에 대한 상세 속성 정보 DataFrame 로딩
movies=pd.read_csv('/content/drive/MyDrive/data/ml-latest-small/movies.csv')

# userId=9의 movieId 데이터를 추출해 movieId=42 데이터가 있는지 확인
movieIds=ratings[ratings['userId']==9]['movieId']

if movieIds[movieIds==42].count()==0:
  print('사용자 아이디 9는 영화 아이디 42의 평점 없음')

print(movies[movies['movieId']==42])

사용자 아이디 9는 영화 아이디 42의 평점 없음
    movieId                   title              genres
38       42  Dead Presidents (1995)  Action|Crime|Drama


* 추천 예상 평점은 predict() 메서드 이용
  + 입력해주는 값은 모두 문자열 값이어야 한다

In [19]:
uid=str(9)
iid=str(42)

pred=algo.predict(uid, iid, verbose=True)

user: 9          item: 42         r_ui = None   est = 3.13   {'was_impossible': False}


* 사용자가 평점을 매기지 않은 전체 영화를 추출한 뒤에 예측 평점 순으로 영화 추천

In [20]:
# 사용자가 아직 평점을 매기지 않은 영화 정보 반환
def get_unseen_surprise(ratings, movies, userId):
  #입력값으로 들어온 userId에 해당하는 사용자가 평점을 매긴 모든 영화를 리스트로 생성
  seen_movies=ratings[ratings['userId']==userId]['movieId'].tolist()

  #모든 영화의 movieId를 리스트로 생성
  total_movies=movies['movieId'].tolist()

  #모든 영화의 movieId 중 이미 평점을 매긴 영화의 movieId를 제외한 후 리스트로 생성
  unseen_movies=[movie for movie in total_movies if movie not in seen_movies]
  print('평점 매긴 영화 수:', len(seen_movies), '추천 대상 영화 수:', len(unseen_movies), '전체 영화 수:', len(total_movies))

  return unseen_movies

unseen_movies=get_unseen_surprise(ratings, movies, 9)

평점 매긴 영화 수: 46 추천 대상 영화 수: 9696 전체 영화 수: 9742


* 학습된 추천 알고리즘 클래스의 SVD를 이용해 높은 예측 평점을 가진 순으로 영화 추천
  + 함수 생성 - 인자로 학습이 완료된 추천 알고리즘 객체, 추천 대상 사용자 아이디, 추천 대상 영화의 리스트 객체, 추천 상위 N개 개수

In [21]:
def recomm_movie_by_surprise(algo, userId, unseen_movies, top_n=10):
  #알고리즘 객체의 predict() 메서드를 평점이 없는 영화에 반복 수행한 후 결과를 list로 저장
  predictions=[algo.predict(str(userId), str(movieId)) for movieId in unseen_movies]

  #predictions list 객체는 surprise의 predictions 객체를 원소로 갖고 있다.
  # [Prediction(uid='9', iid='1', est=3.69), Predictions(uid='9', iid='2', est=2.98),,,]

  #이를 est 값으로 정렬하기 위해 아리애 sortkey_est 함수 정의
  #sortdey_est 함수는 list 객체의 sort() 함수의 키 값으로 사용되어 정렬 수행
  def sortkey_est(pred):
    return pred.est

  #sortkey_est() 반환값의 내림차순으로 정렬 수행하고 top_n개의 최상위 값 추출
  predictions.sort(key=sortkey_est, reverse=True)
  top_predictions=predictions[:top_n]

  #top_n 으로 추출된 영화의 정보 추출. 영화 아이디, 추천 예상 평점, 제목 추출
  top_movie_ids=[int(pred.iid) for pred in top_predictions]
  top_movie_rating=[pred.est for pred in top_predictions]
  top_movie_titles=movies[movies.movieId.isin(top_movie_ids)]['title']
  top_movie_preds=[ (id, title, rating) for id, title, rating in zip(top_movie_ids, top_movie_titles, top_movie_rating)]

  return top_movie_preds

unseen_movies=get_unseen_surprise(ratings, movies, 9)
top_movie_preds=recomm_movie_by_surprise(algo, 9, unseen_movies, top_n=10)

print('### top-10 추천 영화 리스트 ###')
for top_movie in top_movie_preds:
  print(top_movie[1], ':', top_movie[2])

평점 매긴 영화 수: 46 추천 대상 영화 수: 9696 전체 영화 수: 9742
### top-10 추천 영화 리스트 ###
Usual Suspects, The (1995) : 4.306302135700814
Star Wars: Episode IV - A New Hope (1977) : 4.281663842987387
Pulp Fiction (1994) : 4.278152632122759
Silence of the Lambs, The (1991) : 4.226073566460876
Godfather, The (1972) : 4.1918097904381995
Streetcar Named Desire, A (1951) : 4.154746591122657
Star Wars: Episode V - The Empire Strikes Back (1980) : 4.122016128534504
Star Wars: Episode VI - Return of the Jedi (1983) : 4.108009609093436
Goodfellas (1990) : 4.083464936588478
Glory (1989) : 4.07887165526957


## 09. 정리


---

**추천 시스템** : 기업 애플리케이션에서 매우 중요한 위치 차지

1. 콘텐츠 기반 필터링 : 아이템을 구성하는 여러 가지 콘텐츠 중 사용자가 좋아하는 콘텐츠를 필터링하여 이에 맞는 아이템을 추천하는 방식

2. 협업 필터링
  * 최근접 이웃 협업 필터링 - 사용자 기반(사용자-사용자)과 아이템 기반(아이템-아이템)
  * 잠재 요인 협업 필터링

* Surprise : 파이썬의 추천 시스템 패키지